In [4]:
import math

grid = [['','',''],
        ['','',''],
        ['','','']]

#grid resets before every game
def resetGrid():
  for i in range(0, len(grid)):
    for j in range(0, len(grid[i])):
      grid[i][j] = ''

#displays grid
#for mode = 'single' -> displays grid with numbers in empty spaces
#for mode = 'ai' -> displays grid with spaces
def displayGrid(board, mode):
  #counter for displaying appropriate count number on empty space
  count=1
  for i in range(0, len(board)):
    for j in range(0, len(board[i])):
      #detect empty space to place count value or space
      if board[i][j] == '':
        #only last row entry will be different
        if j!=len(board[i])-1:
          if mode == 'single':
            print(count,' | ', end = '')
          else:
            print('   | ', end = '')
        else:
          if mode == 'single':
            print(count)
          else:
            print()
        count+=1
      #prints X or O turn
      else:
        if j!=len(board[i])-1:
          print(board[i][j],' | ', end = '')
        else:
          print(board[i][j])
    #row separators
    if i!=len(board)-1:
      print('---+----+---')
    else:
      print()

#counts depth i.e empty spaces in grid for minimax()
def countDepth(board):
  count=0
  for i in range(0, len(board)):
    for j in range(0, len(board[i])):
      if board[i][j] != 'X' and board[i][j] != 'O':
        count+=1
  return count

#checks possible goal states and returns which player won, otherwise returns -1
def goalTest(board):
  winningCondition = [[(0,0),(1,0),(2,0)],
                      [(0,1),(1,1),(2,1)],
                      [(0,2),(1,2),(2,2)],
                      [(0,0),(0,1),(0,2)],
                      [(1,0),(1,1),(1,2)],
                      [(2,0),(2,1),(2,2)],
                      [(0,0),(1,1),(2,2)],
                      [(0,2),(1,1),(2,0)]]

  for condition in winningCondition:
    pair1 = condition[0]
    pair2 = condition[1]
    pair3 = condition[2]
    if board[pair1[0]][pair1[1]] == board[pair2[0]][pair2[1]] == board[pair3[0]][pair3[1]] and board[pair1[0]][pair1[1]] != '':
      return board[pair1[0]][pair1[1]]
  return -1

#returns appropriate utility value for leaf nodes
def checkWinner(board, winner):
  if winner == False:
    winner = goalTest(board)

  #main player wins
  if winner=='X':
    return 1 
  #opponent wins
  elif winner == 'O':
    return -1
  #draw
  else:
    return 0

#shuffles turn for minimax()
def shuffleTurn(turn):
  if turn == 'X':
    return 'O'
  else:
    return 'X'

#utility function that detects best possible next move
def bestMove(turn):
  bestScore = -math.inf
  move=(-1,-1)
  for i in range(0, len(grid)):
    for j in range(0, len(grid[i])):
      if grid[i][j] == '':
        grid[i][j] = turn
        #maximizingPlayer is False -> bestMove() method acts as maximizingPlayer to get maximum benefit output
        eval = minimax(turn, grid, countDepth(grid), -math.inf, math.inf, False)
        grid[i][j] = ''
        if bestScore<eval:
          bestScore = eval
          move = (i,j)
  #grid updated
  grid[move[0]][move[1]] = turn

#minimax algo for choosing possible good move
def minimax(turn, board, depth, alpha, beta, maximizingPlayer):
  #goalTest is checked as base case to return appropriate values
  # 1: X winner  --  -1: O winner  --  0: Draw
  if goalTest(board) == 'O':
    return -1
  if goalTest(board) == 'X':
    return 1
  if depth == 0:
    return checkWinner(board, False)

  if maximizingPlayer:
    maxEval = -math.inf
    flag = False
    for i in range(0, len(board)):
      for j in range(0, len(board[i])):
        if board[i][j] == '':
          board[i][j] = turn
          eval = minimax(turn, board, depth-1, alpha, beta, False)
          board[i][j] = ''
          maxEval = max(maxEval, eval)
          alpha = max(alpha, eval)
          if beta <= alpha:
            flag = True
            break
      if flag:
        break
    return maxEval
  else:
    minEval = math.inf
    flag = False
    for i in range(0, len(board)):
      for j in range(0, len(board[i])):
        if board[i][j] == '':
          board[i][j] = shuffleTurn(turn)
          eval = minimax(turn, board, depth-1, alpha, beta, True)
          board[i][j] = ''
          minEval = min(minEval, eval)
          beta = min(beta, eval)
          if beta <= alpha:
            flag = True
            break
      if flag:
        break
    return minEval

#userInput method by calculating appropriate count value and matching it with user's input count value
def userInput(board, p2):
  isValid = False
  while isValid==False:
    turn = int(input("Your Turn: "))
    count=0
    move=(-1,-1)
    for i in range(0, len(board)):
      for j in range(0, len(board[i])):
        #counter increments and indices are saved
        if board[i][j] == '' and count!=turn:
          count+=1
          move = (i,j)
      
      #when turn found, loop terminates
      if count == turn:
        isValid = True
        break
    #otherwise wrong move is detected
    if isValid==False:
      print("Enter valid input!")
  
  #grid updated with player's move
  grid[move[0]][move[1]] = p2

#main game method
def gamePlay():
  p1 = 'X'
  p2 = 'O'
  flag = True

  while flag:
    resetGrid()
    print("+---------------------------+")
    print("         Tic Tac Toe")
    print("+---------------------------+")
    print("  Press 1: Single Player")
    print("  Press 2: AI vs AI")
    print("  Press 0: Exit")
    print("+---------------------------+")
    choice = int(input("--> "))

    #Exit Mode
    if choice == 0:
      flag = False
    #Single Player Mode
    elif choice == 1:
      displayGrid(grid, 'single')
      for i in range(0,10):
        #goalTest
        result = goalTest(grid)
        depth = countDepth(grid)
        if result == -1 and depth!=0:
          if i==0 or i == 2 or i == 4 or i == 6 or i == 8:
            print("Turn:",p2)
            userInput(grid, p2)
            displayGrid(grid, 'single')
          else:
            print("Turn:",p1)
            #utility function
            bestMove(p1)
            #updated result
            displayGrid(grid, 'single')   
        else:
          winner = checkWinner(grid, result)
          if winner == 0:
            print("Game tied!")
            break
          else:
            print(result,"won!")
            break
    #AI vs AI mode
    elif choice == 2:
      displayGrid(grid, 'single')
      for i in range(0,10):
        #goaltest
        result = goalTest(grid)
        depth = countDepth(grid)
        if result == -1 and depth!=0:
          if i==0 or i == 2 or i == 4 or i == 6 or i == 8:
            print("Turn:",p1)
            #utility function
            bestMove(p1)
            #updated result
            displayGrid(grid, 'ai') 
          else:
            print("Turn:",p2)
            #utility function
            bestMove(p2)
            #updated result
            displayGrid(grid, 'ai')  
        else:
          winner = checkWinner(grid, result)
          if winner == 0:
            print("Game tied!")
            break
          else:
            print(result,"won!")
          break
    #Invalid Choice Detected
    else:
      print("Invalid choice!")

gamePlay()



+---------------------------+
         Tic Tac Toe
+---------------------------+
  Press 1: Single Player
  Press 2: AI vs AI
  Press 0: Exit
+---------------------------+
--> 2
1  | 2  | 3
---+----+---
4  | 5  | 6
---+----+---
7  | 8  | 9

Turn: X
X  |    | 
---+----+---
   |    | 
---+----+---
   |    | 

Turn: O
X  | O  | 
---+----+---
   |    | 
---+----+---
   |    | 

Turn: X
X  | O  | 
---+----+---
X  |    | 
---+----+---
   |    | 

Turn: O
X  | O  | O
---+----+---
X  |    | 
---+----+---
   |    | 

Turn: X
X  | O  | O
---+----+---
X  | X  | 
---+----+---
   |    | 

Turn: O
X  | O  | O
---+----+---
X  | X  | O
---+----+---
   |    | 

Turn: X
X  | O  | O
---+----+---
X  | X  | O
---+----+---
X  |    | 

X won!
+---------------------------+
         Tic Tac Toe
+---------------------------+
  Press 1: Single Player
  Press 2: AI vs AI
  Press 0: Exit
+---------------------------+
--> 1
1  | 2  | 3
---+----+---
4  | 5  | 6
---+----+---
7  | 8  | 9

Turn: O
Your Turn: 1
O  | 1  